### Imports

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd
import numpy as np
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.analyses import Pf1D
from neuropy.analyses import Decode1d
import subjects

### Decoding re-maze with maze template

In [ ]:
sessions = (
    subjects.nsd.ratSday2
    + subjects.nsd.ratVday1
    + subjects.nsd.ratUday2
    + subjects.sd.ratSday3
    + subjects.sd.ratVday2
    + subjects.sd.ratUday4
)


In [ ]:
from tqdm import tqdm

score_df = []
dist_replay_df = []
for sub, sess in enumerate(tqdm(sessions)):
    re_maze = sess.paradigm["re-maze"].flatten()
    neurons = sess.neurons.get_neuron_type(neuron_type="pyr")
    pos = sess.maze
    # run = sess.run
    pf = Pf1D(
        neurons=neurons,
        position=pos,
        speed_thresh=5,
        sigma=4,
        grid_bin=2,
        # epochs=run,
        frate_thresh=1,
    )
    pf_neurons = neurons.get_by_id(pf.neuron_ids)
    epochs = sess.pbe.time_slice(re_maze[0], re_maze[1])
    decode = Decode1d(
        neurons=pf_neurons,
        ratemap=pf,
        epochs=epochs,
        bin_size=0.02,
        decode_margin=15,
        nlines=5000,
    )
    jump_distance = [np.abs(np.diff(_)).mean() for _ in decode.decoded_position]

    norm_pos = min_max_scaler(decode.ratemap.xbin_centers)
    decoded_position_mean = np.nanmean(np.hstack(decode.posterior), axis=1)
    pos_bins = np.linspace(0, 1, 50)
    mean_pos = np.interp(pos_bins, norm_pos, decoded_position_mean)

    df_dist = pd.DataFrame(
        dict(
            bins=pos_bins,
            mean_pos=mean_pos,
            name=sess.animal.name,
            grp=sess.tag,
        )
    )

    df = pd.DataFrame(
        dict(
            score=decode.score,
            velocity=decode.velocity,
            jump_distance=jump_distance,
            epoch="re-maze",
            name=sess.animal.name,
            grp=sess.tag,
        )
    )
    score_df.append(df)
    dist_replay_df.append(df_dist)

score_df = pd.concat(score_df, ignore_index=True)
dist_replay_df = pd.concat(dist_replay_df, ignore_index=True)
subjects.GroupData().save(score_df, "replay_re_maze_score")
subjects.GroupData().save(dist_replay_df, "replay_re_maze_position_distribution")


In [ ]:
%matplotlib widget
import seaborn as sns

sns.violinplot(
    data=score_df,
    x="epoch",
    y="score",
    hue="grp",
    split=True,
    inner=None,
    scale="width",
)


### Decoding visualization

In [ ]:
from hfuncs import plot_in_bokeh
import bokeh.plotting as bplot

sess = subjects.sd.ratSday3[0]
post = sess.paradigm["post"].flatten()
replay_pbe = sess.remaze_replay_pbe.to_dataframe()
starts = sess.remaze_replay_pbe.starts
wcorr, scores, posteriors = [], [], []
for i, d in enumerate(["up", "down"]):
    scores.append(replay_pbe[d + "_percentile_score"].values)
    posteriors.extend(sess.replay_pbe.metadata[d + "_posterior"])
    wcorr.append(replay_pbe[d + "_score"].values)

wcorr = np.concatenate(wcorr)
scores = np.concatenate(scores)
starts = np.tile(starts, (2,)) - post[0]
scores_color = np.where((scores >= 95) | (scores <= 5), scores, 50)

n_epochs = len(starts)
annotate_dict = dict(evt=np.arange(n_epochs), wcorr=wcorr[:n_epochs])
bplot.output_notebook()
# bplot.output_file(subjects.figpath_sd/'test_wake_decoding.html')
p = plot_in_bokeh(
    starts[:n_epochs]/3600,
    scores[:n_epochs],
    [posteriors[_] for _ in range(n_epochs)],
    annotate=annotate_dict,
    color_by=scores_color[:n_epochs],
    palette="jet",
    size=8,
)
# # p.line(pos.time,pos.x/350 + 0.2,line_width=2,color='black')
bplot.show(p)


### Distribution of wcorr

In [ ]:
sessions = subjects.remaze_sess()

In [ ]:
from replay_funcs import get_jump_distance

replay_df = []
shuffle_df=[]
for sub, sess in enumerate(sessions):

    replay_pbe_df = sess.remaze_replay_pbe.to_dataframe()
    metadata = sess.remaze_replay_pbe.metadata

    wcorr_up = replay_pbe_df["up_score"].values
    percentile_up = replay_pbe_df["up_percentile_score"].values
    shuffle_up = metadata['up_shuffle_score']
    jd_up = get_jump_distance(metadata['up_posterior'],'mean')


    wcorr_down = -replay_pbe_df["down_score"].values
    percentile_down = 100 - replay_pbe_df["down_percentile_score"].values
    shuffle_down = metadata['down_shuffle_score']
    jd_down = get_jump_distance(metadata['down_posterior'],'mean')

    best_bool = np.abs(wcorr_up) > np.abs(wcorr_down)

    wcorr = np.zeros(len(replay_pbe_df))
    wcorr[best_bool] = wcorr_up[best_bool]
    wcorr[~best_bool] =wcorr_down[~best_bool]

    percentile = np.zeros(len(replay_pbe_df))
    percentile[best_bool] = percentile_up[best_bool]
    percentile[~best_bool] = percentile_down[~best_bool]

    shuffles = np.zeros((1000,len(replay_pbe_df)))
    shuffles[:,best_bool] =shuffle_up[:,best_bool]
    shuffles[:,~best_bool] =shuffle_down[:,~best_bool]

    jd = np.zeros_like(wcorr_up)
    jd[best_bool] = np.abs(jd_up[best_bool])
    jd[~best_bool] = np.abs(jd_down[~best_bool])



    df = pd.DataFrame(
        dict(
            zt="remaze",
            wcorr=wcorr,
            perc = percentile,
            jd =jd,
            animal=sub,
            grp=sess.tag,
        )
    )

    df_shuff = pd.DataFrame(
        dict(
            zt="remaze",
            shuffle_scores=shuffles.flatten(),
            animal=sub,
            grp=sess.tag,
        )
    )


    replay_df.append(df)
    shuffle_df.append(df_shuff)


replay_df = pd.concat(replay_df, ignore_index=True)
shuffle_df = pd.concat(shuffle_df, ignore_index=True)
# subjects.GroupData().save(replay_df, "remaze_replay_wcorr_dist")


In [ ]:
fig = plotting.Fig(grid=(8,5))

colors = subjects.colors_sd(1.2)
bins,step = np.linspace(-1,1,50,retstep=True)
bin_cntr = bins[:-1]+step/2

for g,grp in enumerate(['NSD','SD']):
    wcorrs = replay_df[replay_df['grp']==grp].wcorr.values
    counts = np.histogram(wcorrs,bins=bins)[0]
    prob = counts/len(wcorrs)


    ax = fig.subplot(fig.gs[0,0])
    ax.step(bin_cntr,prob,color=colors[g],where='mid')
ax.set_xlabel('Correlation')

bins,step = np.linspace(0,100,20,retstep=True)
bin_cntr = bins[:-1]+step/2

for g,grp in enumerate(['NSD','SD']):
    perc = replay_df[replay_df['grp']==grp].perc.values
    counts = np.histogram(perc,bins=bins)[0]
    prob = counts/len(perc)

    ax = fig.subplot(fig.gs[0,1])
    ax.step(bin_cntr,prob,color=colors[g],where='mid')
ax.set_xlabel('Percentile')

bins,step = np.linspace(0,0.5,50,retstep=True)
bin_cntr = bins[:-1]+step/2
for g,grp in enumerate(['NSD','SD']):
    jds = replay_df[replay_df['grp']==grp].jd.values
    counts = np.histogram(jds,bins=bins)[0]
    prob = counts/len(jds)


    ax = fig.subplot(fig.gs[0,2])
    ax.step(bin_cntr,prob,color=colors[g],where='mid')
ax.set_xlabel('Jump distance')


fig.savefig(subjects.figpath_sd/'remaze_replay')


### Significant forward and reverse replay during remaze 

In [ ]:
sessions = subjects.remaze_sess()

In [ ]:
replay_df = []

for sub, sess in enumerate(sessions):

    replay_pbe_df = sess.remaze_replay_pbe.to_dataframe()
    metadata = sess.remaze_replay_pbe.metadata

    wcorr_up = replay_pbe_df["up_score"].values
    seq_score_up = replay_pbe_df["up_sequence_score"].values
    percentile_up = replay_pbe_df["up_percentile_score"].values
    wcorr_down = -replay_pbe_df["down_score"].values
    seq_score_down = replay_pbe_df["down_sequence_score"].values
    percentile_down = 100 - replay_pbe_df["down_percentile_score"].values

    best_bool = np.abs(wcorr_up) > np.abs(wcorr_down)

    wcorr = np.zeros(len(replay_pbe_df))
    wcorr[best_bool] = wcorr_up[best_bool]
    wcorr[~best_bool] = wcorr_down[~best_bool]

    seq_score = np.zeros(len(replay_pbe_df))
    seq_score[best_bool] = seq_score_up[best_bool]
    seq_score[~best_bool] = seq_score_down[~best_bool]

    percentile = np.zeros(len(replay_pbe_df))
    percentile[best_bool] = percentile_up[best_bool]
    percentile[~best_bool] = percentile_down[~best_bool]

    prop_forward = np.count_nonzero(
        (wcorr > 0) & ((percentile < 2.5) | (percentile > 97.5))
    ) / len(wcorr)

    prop_reverse =np.count_nonzero(
        (wcorr < 0) & ((percentile < 2.5) | (percentile > 97.5))
    ) / len(wcorr)

 

    df = pd.DataFrame(
        dict(
            # zt="remaze",
            replay_order=["forward", "reverse"],
            prop=[prop_forward, prop_reverse],
            animal=sub,
            grp=sess.tag,
        )
    )
    replay_df.append(df)


replay_df = pd.concat(replay_df, ignore_index=True)
subjects.GroupData().save(replay_df, "remaze_replay_wcorr")


In [ ]:
# _,ax = plt.subplots(1,1)
# sns.barplot(data=replay_df,x='zt',y='perc',hue='grp',ax=ax, ci =99)

_, axs = plt.subplots(1,3)

df = replay_df
# d1 = df[df["grp"] == "NSD"].seq_score.values
# d2 = df[df["grp"] == "SD"].seq_score.values
sns.boxplot(
    data=replay_df,
    x="replay_order",
    y='prop',
    hue="grp",
    ax=axs[0],
    palette=subjects.colors_sd(1.3)
    # dodge=True,
    # join=False,
    # width='area',
    # stat="probability",
    # common_norm=False,
    # cumulative=True,
    # showfliers=False,
)
# print(stats.ttest_ind(d1, d2,alternative='greater'))


### Jump distance and wcorr during remaze SD vs NSD

In [ ]:
sessions = subjects.remaze_sess()

In [ ]:
from replay_funcs import get_jump_distance

hist_all = []
for sub, sess in enumerate(sessions):

    replay_pbe_df = sess.remaze_replay_pbe.to_dataframe()
    metadata = sess.remaze_replay_pbe.metadata

    wcorr_up = replay_pbe_df["up_score"].values
    seq_score_up = replay_pbe_df["up_sequence_score"].values
    percentile_up = replay_pbe_df["up_percentile_score"].values
    jd_up = get_jump_distance(metadata['up_posterior'])

    wcorr_down = -replay_pbe_df["down_score"].values
    seq_score_down = replay_pbe_df["down_sequence_score"].values
    percentile_down = 100 - replay_pbe_df["down_percentile_score"].values
    jd_down = get_jump_distance(metadata['down_posterior'])


    best_bool = np.abs(wcorr_up)>np.abs(wcorr_down)

    wcorr = np.zeros_like(wcorr_up)
    wcorr[best_bool] = np.abs(wcorr_up[best_bool])
    wcorr[~best_bool] = np.abs(wcorr_down[~best_bool])

    jd = np.zeros_like(wcorr_up)
    jd[best_bool] = np.abs(jd_up[best_bool])
    jd[~best_bool] = np.abs(jd_down[~best_bool])


    jd_bins = np.arange(0,1.1,0.1)
    wcorr_bins = np.arange(0,1.1,0.1)

    hist_2d = np.histogram2d(jd,wcorr,bins=[jd_bins,wcorr_bins])[0]
    hist_all.append(hist_2d)


In [ ]:
_,axs = plt.subplots(5,2)
axs = axs.reshape(-1)

for i in range(9):
    axs[i].pcolormesh(hist_all[i],cmap='bwr')
    

In [ ]:
_,ax = plt.subplots()
arr = metadata['up_posterior'][0]


ax.imshow(arr,aspect='auto')

In [ ]:
np.max(np.diff(np.argmax(arr,axis=0))*(1/arr.shape[0]))

In [ ]:
# _,ax = plt.subplots(1,1)
# sns.barplot(data=replay_df,x='zt',y='perc',hue='grp',ax=ax, ci =99)

_, axs = plt.subplots()

df = replay_df
# d1 = df[df["grp"] == "NSD"].seq_score.values
# d2 = df[df["grp"] == "SD"].seq_score.values
sns.boxplot(
    data=replay_df,
    x="replay_order",
    y='prop',
    hue="grp",
    ax=axs,
    # dodge=True,
    # join=False,
    # width='area',
    # stat="probability",
    # common_norm=False,
    # cumulative=True,
    # showfliers=False,
)
# print(stats.ttest_ind(d1, d2,alternative='greater'))
